# Common utitities for all production alphas

In [ ]:
function get_recommendee_username()
    dir = "../../data/recommendations"
    open("$dir/recommendee_username.txt") do f
        read(f, String)
    end    
end;

In [ ]:
name = "../../data/recommendations/$(get_recommendee_username())/alphas/$source"
residual_alphas = [];

In [ ]:
using DataFrames

In [ ]:
@nbinclude("../TrainingAlphas/Alpha.ipynb");

In [ ]:
function get_recommendee_list()
    username = get_recommendee_username()
    dir = "../../data/recommendations"
    file = "$dir/$username/splits.jld2"
    load(file)["user_anime_list"]
end

function get_implicit_list()
    username = get_recommendee_username()
    dir = "../../data/recommendations"
    file = "$dir/$username/splits.jld2"
    load(file)["implicit_anime_list"]    
end

function get_split(split)
    @assert split in
            ["training", "validation", "test", "recommendee", "recommendee_inference"]
    file = "../../data/splits/splits.jld2"
    if split == "recommendee"
        return get_recommendee_list()
    elseif split == "recommendee_inference"
        df = get_recommendee_list()
        return RatingsDataset(
            fill(df.user[1], num_items()),
            collect(1:num_items()),
            zeros(num_items()),
        )
    elseif split == "training"
        return cat(load(file, split), get_recommendee_list())
    end
    load(file, split)
end;

In [ ]:
function recommendee_alpha_basepath()
    dir = "../../data/recommendations"
    username = get_recommendee_username()
    mkpath("$dir/$username/alphas")    
end

In [ ]:
function write_recommendee_alpha(model; outdir = source)
    items = 1:num_items()
    training = get_recommendee_list()
    users = fill(training.user[1], length(items))
    alpha = RatingsDataset(users, items, model(items))

    outdir = mkpath("$(recommendee_alpha_basepath())/$outdir")
    save("$outdir/alpha.jld2", Dict("alpha" => alpha))
end;

In [ ]:
function read_recommendee_alpha(alpha)
    load("$(recommendee_alpha_basepath())/$alpha/alpha.jld2")["alpha"]
end;

function get_alpha(alpha, split)
    @assert split in
            ["training", "validation", "test", "recommendee", "recommendee_inference"]
    file = "../../data/alphas/$(alpha)/predictions.jld2"
    if split in ["recommendee", "training"]
        mask = get_split("recommendee").item
        df = read_recommendee_alpha(alpha)
        df = RatingsDataset(df.user[mask], df.item[mask], df.rating[mask])
        return split == "recommendee" ? df : cat(load(file, split), df)
    elseif split == "recommendee_inference"
        return read_recommendee_alpha(alpha)
    end
    load(file, split)
end;

In [ ]:
function reset_recommenndee_alphas()
    rm(recommendee_alpha_basepath(), recursive=true)
end;

In [ ]:
function should_compute(alpha)
    dir = recommendee_alpha_basepath() 
    # see if we've already computed this alpha
    if isfile("$dir/alpha/alpha.jld2")
        return false
    end
    
    # see if we have the dependencies necessary to compute the alpha
    residual_alphas = missing
    for key in ["residual_alphas", "training_residuals"]
        if key in keys(read_params(alpha))
            residual_alphas = read_params(alpha)[key]
        end
    end
    if ismissing(residual_alphas)
        throw(ArgumentError("Could not find residual alphas for $alpha"))
    end
    for α in residual_alphas
        if !isfile("$dir/$α/alpha.jld2")
            return false
        end
    end
    
    # we're good to go!
    true
end;